# `Debug`
----

In [1]:
from typing import Callable, TypeVar, Any
import threading

''' CUSTOM DEBUG TOOL FROM MY OWN LIL' LIBRARY OF HELPERS '''

__COLORS = {
        (_DEBUG := "red"): "\033[91m",
        (_HEADING := "green"): "\033[92m",
        (_UPDATE :="blue"): "\033[94m",
        (_RESET := "reset"): "\033[0m"
    }

# thread-local storage to track nested debug states
bugs = _debug_stack = threading.local()
F = TypeVar('F', bound = Callable) # generic function type

def debug(*, enabled: bool = False) -> Callable[[F], F]:
    import functools
    ''' Decorator that enables or disables debug prints inside the wrapped function. '''

    def decorator(func: F) -> F:
        @functools.wraps(func)
        def wrapper(*args: Any, **kwargs: Any) -> Any:

            if not hasattr(_debug_stack, 'state'):
                _debug_stack.nest = [] # init debug state if missing

            _debug_stack.nest.append(enabled) # push current function's debug flag

            if enabled:
                print(f'\n{__COLORS.get(_HEADING)}[START] {__COLORS[_RESET]}')
                print(f'{__COLORS.get(_DEBUG)}[DEBUG] {__COLORS["reset"]}' + 
                      f'{__COLORS.get(_UPDATE)}Calling "{func.__name__}"...{__COLORS.get(_RESET)}')
            
            try:
                result = func(*args, **kwargs)  # call actual function so behavior is as intended
            finally:
                _debug_stack.nest.pop()
            
            if enabled:
                print(f'{__COLORS.get(_DEBUG)}[DEBUG] {__COLORS[_RESET]}' + 
                      f'{__COLORS.get(_UPDATE)}Function "{func.__name__}" exited...{__COLORS.get(_RESET)}')
                print(f'{__COLORS.get(_HEADING)}[END] {__COLORS[_RESET]}\n')         
            
            return result  # make sure the function still behaves as intended
        return wrapper # return wrapped function
    return decorator  # returns the decorator

# `Show Nested Structures`
----

In [ ]:
from numpy.typing import NDArray
from typing import TypeVar
from collections.abc import Sequence, Set, Mapping
import numpy as np

'''
# NOTE:
    'bound': restrict to subclasses
    'constraints': restrict to EXACT types
'''

T = TypeVar("T", bound = Sequence | Set | Mapping | NDArray)

def _show(data: T, nested: int = 0, depth: int = 5, head: bool = True) -> None:
    """
    Recursively prints the first 5 elements of each nested structure.

    Parameters:
    -----------
    data : T
        The input data structure (list, tuple, dict, NumPy array, etc.).
    depth : int
        The current depth of recursion (default is 0).
    max_depth : int
        The maximum depth to explore to prevent infinite recursion.
    """

    tab = "" if head else "\t" * nested  # indent for readability

    if nested >= depth:
        print(f"{tab}[...]")
        return
    
    _type = type(data).__name__.upper()


    # DICTIONARIES
    if isinstance(data, dict):  

        for k, v in list(data.items())[:depth]:
            print(f"{tab}[{repr(k)}]({_type})({len(data)}):")
            _show(v, nested + 1, depth, False)
        
        if len(data) > depth:
            print(f'\t{tab}[...]')

    # SEQUENCES
    elif isinstance(data, (Sequence, Set)) and not isinstance(data, str): 
        print(f"{tab}({_type})({len(data)}):")

        for item in list(data)[:depth]: # convert set to list to avoid slicing errors
            _show(item, nested + 1, depth, False)
        
        if len(data) > depth:
            print(f'\t{tab}[...]')

    # NP.NDARRAYS
    elif isinstance(data, np.ndarray):
        print(f"{tab}({_type}){data.shape}:")

        if data.ndim > 2:
            raise ValueError("Only accepts 1-2 dimensional arrays")
        elif data.ndim == 1:
            for row in data[:depth]:
                print(f'\t{tab}{row}')
        else:
            for row in data[:depth]:
                print(f'\t{tab}{repr(row[:depth])}')

        if len(data) > depth:
                print(f'\t{tab}[...]')

    # BASE CASE
    # PRIMITIVE DATA TYPE
    else:
        print(f"{tab}{repr(data)}")  # otherwise, just print it


if __name__ == "__main__":
    import numpy as np

    test_data = {
        "bananas": [[1, 2, 3], [4, 5, 50, 60, 70, 0, 7], [8, 9, 10, 11, 12]],
        "xray": ([13, 14, 15], (16, 17, 18)),
        "jimmy": {
            "corelations": [19, 20, 21, 22],
            "lib": {"a": 23, "b": 24, "c": [25, 26, 27]},
        },
        "soccer_score": np.array([[28, 29, 30], [31, 32, 33]])
    }
    
    test_array = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
    _show(test_data)
    _show(test_array)
